In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn. functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

In [65]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 0.01
batch_size = 32
num_epochs = 5

In [5]:
model= torchvision.models.vgg16(pretrained= True)

/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/daniyalkhan/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100.0%


In [26]:
model.features[0]._parameters['weight'].shape

torch.Size([64, 3, 3, 3])

In [28]:
#now we have modify model arch according to out dataset, change classifier layer remove pooling if required and so on

In [ ]:
#we will replace average pooling with something that led input flows as it is
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x
    
model.avgpool= Identity()
model.classifier= nn.Linear(512, 10)

#to cheange specific layer of classifier then model.classifier[0]= nn.Linear(512, 10)

In [41]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [66]:
# transform = transforms.Compose([
#     transforms.ToTensor(),  # Convert to tensor
#     transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Duplicate channel to make it 3-channel
# ])

# Load Data
train_dataset = datasets.CIFAR10(root='data/', train=True, transform=transforms.ToTensor (), download=True)
#data come in dataset/ and it is in numpy so transorfm change to tensor and if not in root then download
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = datasets.CIFAR10(root='data/', train=False, transform=transforms.ToTensor (), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [67]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss ()
optimizer = optim.Adam(model.parameters (), lr=learning_rate)

In [68]:
for epoch in range(num_epochs):
  print(f"epoch {epoch+1}/{num_epochs}")
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    scores = model(data)
    loss = criterion(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer .step()

epoch 1/5
epoch 2/5
epoch 3/5
epoch 4/5
epoch 5/5


In [69]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model) :
  if loader.dataset.train:
    print ('Checking accuracy on training data')
  else:
    print ('Checking accuracy on test data')
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x. to(device=device).squeeze(1)
      y = y.to(device=device)
      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y) .sum()
      num_samples += predictions.size (0)

    print (f'Got {num_correct} / {num_samples} with accuracy {float (num_correct)/float (num_samples) *100: .2f} ')
  model. train()

In [70]:
check_accuracy (train_loader, model)
check_accuracy (test_loader, model)

Checking accuracy on training data
Got 31244 / 50000 with accuracy  62.49 
Checking accuracy on test data
Got 5896 / 10000 with accuracy  58.96 


In [58]:
#now we freeze initial layers and do back prop on last layers

In [ ]:
model= torchvision.models.vgg16(pretrained= True)

/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daniyalkhan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [62]:
#we simple make requires_grad as false so grad doesnot flow to that point
for params in model.parameters():
    params.requires_grad= False

In [64]:
model.avgpool= Identity()
model.classifier= nn.Sequential(nn.Linear(512, 100),
                                nn.ReLU(),
                                nn.Linear(100, 10))
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [71]:
#if model not available in pytorch, then implement that model and download model weights and then load model weights into your created arch